In [1]:
import os
import rasterio
import numpy as np

In [ ]:
def check_band_coverage(directory):
    print(f"\n{'Band':<6} {'Resolution':<10} {'Shape':<15} {'Valid Pixels (%)':<18}")
    print("-" * 60)

    for fname in sorted(os.listdir(directory)):
        if not fname.endswith('.jp2') or '_TCI' in fname:
            continue

        band_code = fname.split('_')[-1].replace('.jp2', '')
        path = os.path.join(directory, fname)

        with rasterio.open(path) as src:
            img = src.read(1)
            total_pixels = img.size
            valid_pixels = np.count_nonzero(img)
            valid_pct = 100 * valid_pixels / total_pixels

            print(f"{band_code:<6} {src.res[0]:<10.1f} {img.shape!s:<15} {valid_pct:<18.2f}")


check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SEJ_20230118T233535.SAFE/GRANULE/L1C_T34SEJ_A032694_20210925T092343/IMG_DATA")
check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SFJ_20230118T233535.SAFE/GRANULE/L1C_T34SFJ_A032694_20210925T092343/IMG_DATA")
check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TEK_20230118T233535.SAFE/GRANULE/L1C_T34TEK_A032694_20210925T092343/IMG_DATA")
check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TFK_20230118T233535.SAFE/GRANULE/L1C_T34TFK_A032694_20210925T092343/IMG_DATA")


Band   Resolution Shape           Valid Pixels (%)  
------------------------------------------------------------
B01    60.0       (1830, 1830)    100.00            
B02    10.0       (10980, 10980)  100.00            


KeyboardInterrupt: 

In [ ]:
BANDS_10M = ['B02', 'B03', 'B04', 'B08']
BANDS_20M = ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12']
BANDS_60M = ['B01', 'B09', 'B10']
ALL_BANDS = BANDS_10M + BANDS_20M + BANDS_60M
TARGET_SHAPE = (10980, 10980)

def pansharpen_to_10m_and_save(directory, output_tiff="pansharpened.tif"):
    band_paths = {}
    for fname in os.listdir(directory):
        if fname.endswith('.jp2') and '_TCI' not in fname:
            band_code = fname.split('_')[-1].replace('.jp2', '')
            if band_code in ALL_BANDS:
                band_paths[band_code] = os.path.join(directory, fname)

    missing = [b for b in ALL_BANDS if b not in band_paths]
    if missing:
        raise ValueError(f"Missing bands: {missing}")

    # Use metadata from a 10m band as reference
    with rasterio.open(band_paths[BANDS_10M[0]]) as ref_src:
        reference_meta = ref_src.meta.copy()
        reference_meta.update({
            "count": len(ALL_BANDS),
            "height": TARGET_SHAPE[0],
            "width": TARGET_SHAPE[1],
            "driver": "GTiff"
        })

    # Order bands as requested
    ordered_bands = BANDS_60M + BANDS_10M + BANDS_20M

    # Create output TIFF and write one band at a time
    with rasterio.open(output_tiff, "w", **reference_meta) as dst:
        for idx, band in enumerate(ordered_bands, start=1):
            print(idx, band)
            with rasterio.open(band_paths[band]) as src:
                img = src.read(1)

                if band in BANDS_10M:
                    sharpened = img  # No resizing needed
                else:
                    sharpened = cv2.resize(
                        img,
                        (TARGET_SHAPE[1], TARGET_SHAPE[0]),
                        interpolation=cv2.INTER_CUBIC
                    )

                dst.write(sharpened, idx)

    print(f"\nSaved pansharpened image to: {output_tiff}")


In [4]:
pansharpen_to_10m_and_save("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SEJ_20230118T233535.SAFE/GRANULE/L1C_T34SEJ_A032694_20210925T092343/IMG_DATA", output_tiff="data/T34SEJ_pansharpened.tif")
pansharpen_to_10m_and_save("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SFJ_20230118T233535.SAFE/GRANULE/L1C_T34SFJ_A032694_20210925T092343/IMG_DATA", output_tiff="data/T34SFJ_pansharpened.tif")
pansharpen_to_10m_and_save("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TEK_20230118T233535.SAFE/GRANULE/L1C_T34TEK_A032694_20210925T092343/IMG_DATA", output_tiff="data/T34TEK_pansharpened.tif")
pansharpen_to_10m_and_save("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TFK_20230118T233535.SAFE/GRANULE/L1C_T34TFK_A032694_20210925T092343/IMG_DATA", output_tiff="data/T34TFK_pansharpened.tif")


1 B01
2 B09
3 B10
4 B02
5 B03
6 B04
7 B08
8 B05
9 B06
10 B07
11 B8A
12 B11
13 B12

Saved pansharpened image to: data/T34SEJ_pansharpened.tif
1 B01
2 B09
3 B10
4 B02
5 B03
6 B04
7 B08
8 B05
9 B06
10 B07
11 B8A
12 B11
13 B12

Saved pansharpened image to: data/T34SFJ_pansharpened.tif
1 B01
2 B09
3 B10
4 B02
5 B03
6 B04
7 B08
8 B05
9 B06
10 B07
11 B8A
12 B11
13 B12

Saved pansharpened image to: data/T34TEK_pansharpened.tif
1 B01
2 B09
3 B10
4 B02
5 B03
6 B04
7 B08
8 B05
9 B06
10 B07
11 B8A
12 B11
13 B12

Saved pansharpened image to: data/T34TFK_pansharpened.tif


In [2]:
def read_pansharpened_tiff(tiff_path):
    with rasterio.open(tiff_path) as src:
        stack = src.read()  # shape will be (bands, height, width)
        print(f"Stack shape: {stack.shape}")
    return stack

stack = read_pansharpened_tiff("data/T34SEJ_pansharpened.tif")
print(stack.shape)
stack = read_pansharpened_tiff("data/T34SFJ_pansharpened.tif")
print(stack.shape)
stack = read_pansharpened_tiff("data/T34TEK_pansharpened.tif")
print(stack.shape)
stack = read_pansharpened_tiff("data/T34TFK_pansharpened.tif")
print(stack.shape)

Stack shape: (13, 10980, 10980)
(13, 10980, 10980)
Stack shape: (13, 10980, 10980)
(13, 10980, 10980)
Stack shape: (13, 10980, 10980)
(13, 10980, 10980)
Stack shape: (13, 10980, 10980)
(13, 10980, 10980)
